In [7]:
def bayesian_polynomial_regression(arrays, batch_info, degree=3, n_samples=1000):
    """
    Perform Bayesian polynomial regression on multiple arrays and assign probabilities
    to each array being a true signal.
    
    Parameters:
    -----------
    arrays : list of np.ndarray
        List of 1x100 arrays containing the data
    batch_info : list
        List of batch labels corresponding to each array
    degree : int
        Degree of polynomial to fit (default: 3)
    n_samples : int
        Number of MCMC samples (default: 1000)
        
    Returns:
    --------
    dict:
        coefficients: np.ndarray - Mean polynomial coefficients
        array_probs: np.ndarray - Probability of each array being true signal
        trace: arviz.InferenceData - MCMC trace for further analysis
    """
    import numpy as np
    import pymc as pm
    import arviz as az
    from sklearn.preprocessing import PolynomialFeatures, StandardScaler
    
    # Convert inputs to numpy arrays
    X = np.arange(100).reshape(-1, 1)
    arrays = np.array(arrays)
    batch_info = np.array(batch_info)
    n_arrays = len(arrays)
    
    # Create polynomial features
    poly = PolynomialFeatures(degree=degree)
    X_poly = poly.fit_transform(X)
    
    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_poly)
    
    # Build Bayesian model
    with pm.Model() as model:
        # Global parameters
        beta = pm.Normal('beta', mu=0, sigma=1, shape=degree+1)
        sigma = pm.HalfNormal('sigma', sigma=1)
        
        # Batch effects
        batch_effect = pm.Normal('batch_effect', mu=0, sigma=0.5, 
                               shape=len(np.unique(batch_info)))
        
        # Signal probability for each array
        array_weight = pm.Beta('array_weight', alpha=2, beta=2, shape=n_arrays)
        
        # Expected value
        mu = pm.Deterministic('mu', pm.math.dot(X_scaled, beta))
        
        # Add batch effects to expected value
        mu_with_batch = mu + batch_effect[batch_info][:, None]
        
        # Likelihood
        for i in range(n_arrays):
            signal = pm.Normal(f'signal_{i}', 
                             mu=mu_with_batch[i], 
                             sigma=sigma, 
                             observed=arrays[i],
                             shape=100)
        
        # Sample from the model
        trace = pm.sample(n_samples, return_inferencedata=True)
    
    # Extract results correctly
    array_probs = trace.posterior['array_weight'].mean(dim=['chain', 'draw']).values
    coef_means = trace.posterior['beta'].mean(dim=['chain', 'draw']).values
    
    # Transform coefficients back to original scale
    final_coefs = coef_means / scaler.scale_[1:]
    
    return {
        'coefficients': final_coefs,
        'array_probs': array_probs,
        'trace': trace
    }

In [8]:
import numpy as np

# Generate sample data
n_arrays = 20
x = np.linspace(0, 1, 100)
true_signal = 3*x**2 - 2*x + 1

arrays = []
batch_info = []

# Generate some clean signals with batch effects
for i in range(n_arrays):
    batch = i % 3
    batch_effect = np.random.normal(batch*0.5, 0.1, 100)
    noise = np.random.normal(0, 0.1, 100)
    signal = true_signal + batch_effect + noise
    arrays.append(signal)
    batch_info.append(batch)

# Add some pure noise arrays
for i in range(5):
    arrays.append(np.random.normal(0, 1, 100))
    batch_info.append(i % 3)

results = bayesian_polynomial_regression(arrays, batch_info)

# Visualize results
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.subplot(121)
plt.plot(results['array_probs'], 'o-')
plt.title('Probability of Each Array Being Signal')
plt.xlabel('Array Index')
plt.ylabel('Probability')

plt.subplot(122)
x_plot = np.linspace(0, 100, 100)
y_pred = np.polyval(results['coefficients'][::-1], x_plot)
plt.plot(x_plot, y_pred, 'r-', label='Fitted Polynomial')
plt.title('Fitted Polynomial')
plt.xlabel('X')
plt.ylabel('Y')
plt.legend()
plt.tight_layout()
plt.show()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta, sigma, batch_effect, array_weight]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 45 seconds.


ValueError: operands could not be broadcast together with shapes (4,) (3,) 

In [9]:
def bayesian_polynomial_regression(arrays, batch_info, degree=3, n_samples=1000):
    """
    Perform Bayesian polynomial regression on multiple arrays and assign probabilities
    to each array being a true signal.
    
    Parameters:
    -----------
    arrays : list of np.ndarray
        List of 1x100 arrays containing the data
    batch_info : list
        List of batch labels corresponding to each array
    degree : int
        Degree of polynomial to fit (default: 3)
    n_samples : int
        Number of MCMC samples (default: 1000)
    """
    import numpy as np
    import pymc as pm
    import arviz as az
    from sklearn.preprocessing import PolynomialFeatures, StandardScaler
    
    # Convert inputs to numpy arrays
    X = np.arange(100).reshape(-1, 1)
    arrays = np.array(arrays)
    batch_info = np.array(batch_info)
    n_arrays = len(arrays)
    
    # Create polynomial features
    poly = PolynomialFeatures(degree=degree)
    X_poly = poly.fit_transform(X)
    
    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_poly)
    
    # Build Bayesian model
    with pm.Model() as model:
        # Global parameters
        beta = pm.Normal('beta', mu=0, sigma=1, shape=degree+1)
        sigma = pm.HalfNormal('sigma', sigma=1)
        
        # Batch effects
        batch_effect = pm.Normal('batch_effect', mu=0, sigma=0.5, 
                               shape=len(np.unique(batch_info)))
        
        # Signal probability for each array
        array_weight = pm.Beta('array_weight', alpha=2, beta=2, shape=n_arrays)
        
        # Expected value
        mu = pm.Deterministic('mu', pm.math.dot(X_scaled, beta))
        
        # Add batch effects to expected value
        mu_with_batch = mu + batch_effect[batch_info][:, None]
        
        # Likelihood
        for i in range(n_arrays):
            signal = pm.Normal(f'signal_{i}', 
                             mu=mu_with_batch[i], 
                             sigma=sigma, 
                             observed=arrays[i],
                             shape=100)
        
        # Sample from the model
        trace = pm.sample(n_samples, return_inferencedata=True)
    
    # Extract results correctly
    array_probs = trace.posterior['array_weight'].mean(dim=['chain', 'draw']).values
    coef_means = trace.posterior['beta'].mean(dim=['chain', 'draw']).values
    
    # Transform coefficients back to original scale
    # No need to scale coefficients as we'll use the standardized features for prediction
    
    return {
        'coefficients': coef_means,
        'array_probs': array_probs,
        'trace': trace,
        'scaler': scaler,
        'poly': poly
    }

def predict_polynomial(results, x):
    """
    Make predictions using the fitted polynomial model
    
    Parameters:
    -----------
    results : dict
        Results from bayesian_polynomial_regression
    x : array-like
        X values to predict
        
    Returns:
    --------
    y_pred : np.ndarray
        Predicted y values
    """
    # Transform x using the same polynomial features and scaling
    X_poly = results['poly'].transform(x.reshape(-1, 1))
    X_scaled = results['scaler'].transform(X_poly)
    
    # Make predictions
    y_pred = np.dot(X_scaled, results['coefficients'])
    return y_pred


In [ ]:
import numpy as np

# Generate sample data
n_arrays = 20
x = np.linspace(0, 1, 100)
true_signal = 3*x**2 - 2*x + 1

arrays = []
batch_info = []

# Generate some clean signals with batch effects
for i in range(n_arrays):
    batch = i % 3
    batch_effect = np.random.normal(batch*0.5, 0.1, 100)
    noise = np.random.normal(0, 0.1, 100)
    signal = true_signal + batch_effect + noise
    arrays.append(signal)
    batch_info.append(batch)

# Add some pure noise arrays
for i in range(5):
    arrays.append(np.random.normal(0, 1, 100))
    batch_info.append(i % 3)

results = bayesian_polynomial_regression(arrays, batch_info)

# Visualize results
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))

# Plot array probabilities
plt.subplot(121)
plt.plot(results['array_probs'], 'o-')
plt.title('Probability of Each Array Being Signal')
plt.xlabel('Array Index')
plt.ylabel('Probability')

# Plot fitted polynomial
plt.subplot(122)
x_plot = np.linspace(0, 100, 100)
y_pred = predict_polynomial(results, x_plot)
plt.plot(x_plot, y_pred, 'r-', label='Fitted Polynomial')
plt.title('Fitted Polynomial')
plt.xlabel('X')
plt.ylabel('Y')
plt.legend()
plt.tight_layout()
plt.show()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta, sigma, batch_effect, array_weight]


Output()